In [1]:
import pandas as pd

df = pd.read_excel('Data Analyst File.xlsx', sheet_name='Admin')

/Users/youweicheng/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
df.head()

,ID,Med,Admin Date,Units
0,1,Med A,2012-07-02,1500
1,1,Med A,2012-07-06,1500
2,1,Med A,2012-07-09,1500
3,1,Med A,2012-07-11,1500
4,1,Med A,2012-07-13,1500


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          2022 non-null   int64         
 1   Med         2022 non-null   object        
 2   Admin Date  2022 non-null   datetime64[ns]
 3   Units       2022 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 63.3+ KB


In [4]:
def trans_date(Series):

    df['year'] = pd.to_datetime(df[Series]).dt.year
    df['month'] = pd.to_datetime(df[Series]).dt.month
    df['day'] = pd.to_datetime(df[Series]).dt.day
    
trans_date('Admin Date')

In [5]:
df[['month', 'Units']] = df[['month', 'Units']].astype('int')


In [6]:
df

,ID,Med,Admin Date,Units,year,month,day
0,1,Med A,2012-07-02,1500,2012,7,2
1,1,Med A,2012-07-06,1500,2012,7,6
2,1,Med A,2012-07-09,1500,2012,7,9
3,1,Med A,2012-07-11,1500,2012,7,11
4,1,Med A,2012-07-13,1500,2012,7,13
...,...,...,...,...,...,...,...
2017,129,Med A,2012-08-27,5200,2012,8,27
2018,129,Med A,2012-08-30,5200,2012,8,30
2019,129,Med A,2012-09-04,5200,2012,9,4
2020,129,Med A,2012-09-06,6500,2012,9,6


In [7]:
# 1. Total number of units in each month for each medication across patients. 

total_number_units_utilized = df.groupby(['Med', 'month'])['Units'].sum()
total_number_units_utilized

Med    month
Med A  7        4303700
       8        4477100
       9         849900
       11         75300
       12         10200
Med B  9            535
       10           393
       11           420
       12             1
Name: Units, dtype: int64

In [8]:
# 2. how many pateints received Med A or Med B in each month from July to Nov

patient_receive_med_a_b = df.groupby(['Med', 'month'])['ID'].sum()
patient_receive_med_a_b

Med    month
Med A  7        55745
       8        53152
       9        11025
       11         246
       12         143
Med B  9         5088
       10        4401
       11        5096
       12         108
Name: ID, dtype: int64

In [9]:
# 3. what's the average total monthly dose per patient for each medication in each month (July to Nov)

average_units_utilized = df.groupby(['Med', 'month'])['Units'].mean()
average_units_utilized

Med    month
Med A  7         5129.558999
       8         5645.775536
       9         5311.875000
       11       10757.142857
       12        3400.000000
Med B  9            7.039474
       10           5.779412
       11           5.600000
       12           1.000000
Name: Units, dtype: float64

In [10]:
average_units_utilized = df.groupby(['Med', 'month'])['ID'].count()
average_units_utilized

Med    month
Med A  7        839
       8        793
       9        160
       11         7
       12         3
Med B  9         76
       10        68
       11        75
       12         1
Name: ID, dtype: int64

In [11]:
# 4. In each month seperately (Sep, Oct, Nov) and also all together across these 3 months,
# how many patients are switched from Med A to Med B? In each month separately (Sep, Oct, Nov)
# how many patients are started on Med B having not been on Med A before

months = [9, 10, 11]
df_4 = df[df['month'].isin(months)]

def switch_to_med_b(group):
    med_a = group[group['Med'] == 'Med A']
    med_b = group[group['Med'] == 'Med B']
    if len(med_a) > 0 and len(med_b) > 0 and med_b.index.min() > med_a.index.max():
        return 1
    return 0

def started_on_med_b(group):
    med_a = group[group['Med'] == 'Med A']
    med_b = group[group['Med'] == 'Med B']
    if len(med_a) == 0 and len(med_b) > 0:
        return 1
    return 0


switched_counts = df_4.groupby('month').apply(lambda group: group.groupby('ID').apply(switch_to_med_b).sum())
print('Patients switched from Med A to Med B per month:')
print(switched_counts)

started_counts = df_4.groupby('month').apply(lambda group: group.groupby('ID').apply(started_on_med_b).sum())
print('\nPatients started on Med B without Mad A per month:')
print(started_counts)

total_switched = df_4.groupby('ID').apply(switch_to_med_b).sum()
print(f'\nTotal patients switched from Med A to Med B across all months: {total_switched}')

/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/2882843451.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  switched_counts = df_4.groupby('month').apply(lambda group: group.groupby('ID').apply(switch_to_med_b).sum())
/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/2882843451.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  switched_counts = df_4.groupby('month').appl

Patients switched from Med A to Med B per month:
month
9     62
10     0
11     0
dtype: int64

Patients started on Med B without Mad A per month:
month
9     14
10    66
11    72
dtype: int64

Total patients switched from Med A to Med B across all months: 67


/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/2882843451.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  started_counts = df_4.groupby('month').apply(lambda group: group.groupby('ID').apply(started_on_med_b).sum())
/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/2882843451.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  started_counts = df_4.groupby('month').apply

In [12]:
a = df[df['Med'] == 'Med A']['Admin Date'].min()
b = df[df['Med'] == 'Med A']['Admin Date'].max()
b - a

Timedelta('155 days 00:00:00')

In [13]:
# 5. In each month separately (Sep, Oct, Nov) and across all 3 months, 
# for patients switched to Med B, what is the average number of weeks the patients were on Med A
# before being switched to Med B
# Assume a week is 7 days and a month is 4.33 weeks

months = [9 ,10 , 11]
df_5 = df[df['month'].isin(months)]


def weeks_on_med_a(group):
    med_a = group[group['Med'] == 'Med A']
    med_b = group[group['Med'] == 'Med B']
    if len(med_a) > 0 and len(med_b) > 0 and med_b.index.min() > med_a.index.max():
        med_a_start = med_a.iloc[0]['Admin Date']
        med_b_start = med_b.iloc[0]['Admin Date']
        weeks_on_med_a = (med_b_start - med_a_start).days / 7
        return weeks_on_med_a
    return 0

avg_weeks_per_month = df_5.groupby('month').apply(lambda group: df_5[df_5['month'] == group.name].groupby('ID').apply(weeks_on_med_a).mean())
print('Average weeks on Med A before switching to B per month:')
print(avg_weeks_per_month)

avg_weeks_total = df.groupby("ID").apply(weeks_on_med_a).mean()
print(f'\nAverage weeks on Med A before switching to B across all months: {avg_weeks_total}')

/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/213018819.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_weeks_per_month = df_5.groupby('month').apply(lambda group: df_5[df_5['month'] == group.name].groupby('ID').apply(weeks_on_med_a).mean())


Average weeks on Med A before switching to B per month:
month
9     1.134647
10    0.000000
11    0.000000
dtype: float64

Average weeks on Med A before switching to B across all months: 6.311184939091915


/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/213018819.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_weeks_per_month = df_5.groupby('month').apply(lambda group: df_5[df_5['month'] == group.name].groupby('ID').apply(weeks_on_med_a).mean())
/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/213018819.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_weeks_per_

In [49]:
# 6. What is the average total monthly dose per patient per month(in patients that swithced) 
# of Med A before switching to Med B
# What is the average total monthly dose per patient
# per month of Med B (in patients that switched - assume Med B dose is for 1 month)

months = [9, 10, 11]
df_6 = df[df['month'].isin(months)]


def monthly_dose(group):
    med_a = group[group['Med'] == 'Med A']
    med_b = group[group['Med'] == 'Med B']
    if len(med_a) > 0 and len(med_b) > 0 and med_b.index.min() > med_a.index.max():
        med_a_monthly_dose = med_a['Units'].sum() * 4.33 / 1.134647
        med_b_monthly_dose = med_b['Units'].sum()
        return pd.Series({'med_a_monthly_dose': med_a_monthly_dose, 'med_b_monthly_dose': med_b_monthly_dose})
    return pd.Series({'med_a_monthly_dose': 0, 'med_b_monthly_dose': 0})

monthly_doses = df.groupby('ID').apply(monthly_dose)
avg_med_a_dose = monthly_doses['med_a_monthly_dose'].mean()
avg_med_b_dose = monthly_doses['med_b_monthly_dose'].mean()

print(f'Average total monthly dose per patient of Med A before switching: {avg_med_a_dose}')
print(f'Average total monthly dose per patient of Med B: {avg_med_b_dose}')



Average total monthly dose per patient of Med A before switching: 248225.2531048775
Average total monthly dose per patient of Med B: 9.713178294573643


/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/4159136882.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_doses = df.groupby('ID').apply(monthly_dose)


In [48]:
# 7. If Medication A cost 1 for 100 units, what is the breakeaven price point for Medication B (per unit of B)?

print(avg_med_a_dose/100/avg_med_b_dose)

255.55512889488585


In [59]:
# 8. How much does the average total monthly dose per patient (Medication A and B) change for patients switched Sep vs Oct vs Nov?

months = [9, 10, 11]
df_6 = df[df['month'].isin(months)]


def monthly_dose(group):
    med_a = group[group['Med'] == 'Med A']
    med_b = group[group['Med'] == 'Med B']
    if len(med_a) > 0 and len(med_b) > 0 and med_b.index.min() > med_a.index.max():
        med_a_monthly_dose = med_a['Units'].sum() * 4.33 / 1.134647
        med_b_monthly_dose = med_b['Units'].sum()
        return pd.Series({'med_a_monthly_dose': med_a_monthly_dose, 'med_b_monthly_dose': med_b_monthly_dose})
    return pd.Series({'med_a_monthly_dose': 0, 'med_b_monthly_dose': 0})

monthly_doses = df.groupby(['month', 'ID']).apply(monthly_dose)
monthly_doses



/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/1994102582.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_doses = df.groupby(['month', 'ID']).apply(monthly_dose)


med_a_monthly_dose  med_b_monthly_dose
month ID                                         
7     1                   0.0                 0.0
      2                   0.0                 0.0
      3                   0.0                 0.0
      4                   0.0                 0.0
      5                   0.0                 0.0
...                       ...                 ...
11    128                 0.0                 0.0
12    6                   0.0                 0.0
      58                  0.0                 0.0
      79                  0.0                 0.0
      108                 0.0                 0.0

[417 rows x 2 columns]

In [60]:
monthly_doses = monthly_doses.reset_index(level=[0,1])
monthly_doses

,month,ID,med_a_monthly_dose,med_b_monthly_dose
0,7,1,0.0,0.0
1,7,2,0.0,0.0
2,7,3,0.0,0.0
3,7,4,0.0,0.0
4,7,5,0.0,0.0
...,...,...,...,...
412,11,128,0.0,0.0
413,12,6,0.0,0.0
414,12,58,0.0,0.0
415,12,79,0.0,0.0


In [61]:
avg_doses_per_month = monthly_doses.groupby('month')[['med_a_monthly_dose', 'med_b_monthly_dose']].mean()
avg_doses_per_month

,med_a_monthly_dose,med_b_monthly_dose
month,,
7,0.000000,0.000000
8,0.000000,0.000000
9,33231.338981,5.149425
10,0.000000,0.000000
11,0.000000,0.000000
12,0.000000,0.000000


In [62]:
# 9. In patients what were switched to Med B, what percent of the 2nd Med B dose (total dose in month following 1st dose)
# was the same as  the 1st Med B dose? Higher than the 1st dose? Lower than the first dose (nut not a zero dose)?
# No dose at all (a zero dose)? (calculate for patients swithced in Sep only, October only, and Sep and Oct together
# assume Med B dose is for 1 month only)


def categorize_dose_change(group):
    med_b = group[group['Med'] == 'Med B']
    if len(med_b) < 2:
        return pd.Series({'same': 0, 'higher': 0, 'lower': 0, 'zero': 0})
    
    first_dose = med_b.iloc[0]['Units']
    second_dose = med_b.iloc[1]['Units']
    
    if first_dose == second_dose:
        return pd.Series({'same': 1, 'higher': 0, 'lower': 0, 'zero': 0})
    elif second_dose > first_dose:
        return pd.Series({'same': 0, 'higher': 1, 'lower': 0, 'zero': 0})
    elif second_dose < first_dose and second_dose > 0:
        return pd.Series({'same': 0, 'higher': 0, 'lower': 1, 'zero': 0})
    else:
        return pd.Series({'same': 0, 'higher': 0, 'lower': 0, 'zero': 1})


sep_df = df[df['month'] == 9]
sep_counts = sep_df.groupby('ID').apply(categorize_dose_change).sum()
sep_totals = sep_counts.sum()
sep_percentages = (sep_counts / sep_totals * 100).fillna(0)
print("Percentages for September:")
print(sep_percentages)


oct_df = df[df['month'] == 10]
oct_counts = oct_df.groupby('ID').apply(categorize_dose_change).sum()
oct_totals = oct_counts.sum()
oct_percentages = (oct_counts / oct_totals * 100).fillna(0)
print("\nPercentages for October:")
print(oct_percentages)


sep_oct_df = df[df['month'].isin([9, 10])]
sep_oct_counts = sep_oct_df.groupby('ID').apply(categorize_dose_change).sum()
sep_oct_totals = sep_oct_counts.sum()
sep_oct_percentages = (sep_oct_counts / sep_oct_totals * 100).fillna(0)
print("\nPercentages for September and October combined:")
print(sep_oct_percentages)

Percentages for September:
same      0.0
higher    0.0
lower     0.0
zero      0.0
dtype: float64

Percentages for October:
same       0.0
higher    50.0
lower     50.0
zero       0.0
dtype: float64

Percentages for September and October combined:
same      59.615385
higher    13.461538
lower     26.923077
zero       0.000000
dtype: float64


/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/1297336521.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sep_counts = sep_df.groupby('ID').apply(categorize_dose_change).sum()
/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/1297336521.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  oct_counts = oct_df.groupby('ID').apply(categorize_dose_change).sum()
/var/folders/w

In [158]:
# 10. For patients that switch from Med A to Med B (question 4), what’s the average LAB B value for these patients when they were on Med A? Med B?


df_lab = pd.read_excel('Data Analyst File.xlsx', sheet_name='Labs')



def switched_to_med_b(group):
    med_a = group[group['Med'] == 'Med A']
    med_b = group[group['Med'] == 'Med B']
    if len(med_a) > 0 and len(med_b) > 0 and med_b.index.min() > med_a.index.max():
        return group['ID'].unique()[0]
    return None


switched_patients = df.groupby('ID').apply(switched_to_med_b).dropna().unique()

print("Patients who switched from Med A to Med B:")
print(switched_patients)

switched_patients = switched_patients.astype('int64')


df_10 = df_lab.groupby(['ID', "LAB_RESULT_CODE"])['LAB_VALUE'].sum()

df_10


    
   








Patients who switched from Med A to Med B:
[  1.   2.   4.   5.   8.   9.  10.  12.  15.  16.  17.  18.  19.  20.
  21.  22.  25.  28.  29.  30.  31.  33.  35.  36.  38.  39.  42.  46.
  47.  48.  49.  52.  53.  54.  57.  60.  63.  65.  66.  67.  69.  73.
  74.  75.  77.  81.  82.  83.  85.  86.  87.  90.  91.  92.  94.  96.
  97.  98.  99. 100. 101. 104. 105. 106. 108. 109. 110. 111. 112. 113.
 114. 116. 117. 118. 119. 121. 122. 125. 126. 128. 129.]


/Users/youweicheng/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/var/folders/wk/sjdjx94s6pzdbsvs0ktmtncr0000gn/T/ipykernel_3262/2847439292.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  switched_patients = df.groupby('ID').apply(switched_to_med_b).dropna().unique()


ID     LAB_RESULT_CODE
1.0    LAB A                                           679621915848845611
       LAB B                 10.710.210.310.310.410.711.011.011.911.111.1
       LAB C                                                  26725406022
2.0    LAB A                                         10791190120416221181
       LAB B              9.611.311.212.012.910.610.010.813.012.012.510.6
                                               ...                       
128.0  LAB B               11.29.611.610.711.212.011.611.210.911.310.19.2
       LAB C                                                   3226272449
129.0  LAB A                                                    709744439
       LAB B                                    11.711.89.89.810.210.09.4
       LAB C                                                       363927
Name: LAB_VALUE, Length: 351, dtype: object